In [ ]:
# Import libraries
import pandas as pd

In [ ]:
# Read data

data = pd.read_excel('keyword_extraction_eval_data.xlsx')
data['pattern_webb'] = data['pattern_webb'].apply(lambda x:list(eval(str(x))))
data['pattern_siahaan'] = data['pattern_siahaan'].apply(lambda x:list(eval(str(x))))
data['pattern_johann'] = data['pattern_johann'].apply(lambda x:list(eval(str(x))))
data['new_pattern'] = data['new_pattern'].apply(lambda x:list(eval(str(x))))
data['expected_result'] = data['expected_result'].apply(lambda x:list(eval(str(x))))
data['expected_result'] = data['expected_result'].apply(lambda x:list(set(x)))

data.head()

,data,expected_result,tokenized,pos_tagged_spacy,lemmatized,pattern_webb,pattern_siahaan,pattern_johann,new_pattern
0,the present invention relates to a method for ...,"[(inputting, target, word), (automatically, ge...","[['present', 'invention', 'relates', 'method',...","[[('present', 'JJ'), ('invention', 'NN'), ('re...","[[('present', 'JJ'), ('invention', 'NN'), ('re...","[(base, word), (mean, clarification), (base, w...","[(base, word), (base, word), (provide, examine...","[(generate, vocabulary, problem), (use, deep, ...","[(generate, vocabulary, problem), (use, deep, ..."
1,"the present disclosure is directed to methods,...","[(provide, different, way, to, prepare, food, ...","[['present', 'disclosure', 'directed', 'method...","[[('present', 'JJ'), ('disclosure', 'NN'), ('d...","[[('present', 'JJ'), ('disclosure', 'NN'), ('d...","[(prepare, food), (provide, scalability), (pro...","[(prepare, food, dish, subsystem), (provide, s...","[(include, robotic, kitchen), (provide, differ...","[(include, robotic, kitchen, calibration), (pr..."
2,an anti-theft method for an oil tank truck bas...,"[(output, an, alarm, signal, to, a, local, ala...","[['anti-theft', 'method', 'oil', 'tank', 'truc...","[[('anti', 'JJ'), ('-', 'JJ'), ('theft', 'JJ')...","[[('anti', 'JJ'), ('-', 'JJ'), ('theft', 'JJ')...","[(base, machine), (follow, step), (acquire, sc...","[(base, machine, learning, comprises), (follow...","[(base, machine), (follow, step), (acquire, sc...","[(base, machine, learning, comprises), (follow..."
3,the invention provides an artificial intellige...,"[(categorize, subsequent), (categorize, event)...","[['invention', 'provides', 'artificial', 'inte...","[[('invention', 'NN'), ('provides', 'VBZ'), ('...","[[('invention', 'NN'), ('provide', 'VBZ'), ('a...","[(categorize, event), (categorize, event), (pr...","[(categorize, event), (categorize, event), (pr...","[(categorize, event), (comprise, first, state)...","[(categorize, event), (comprise, first, state)..."
4,systems and methods for using machine learning...,"[(predict, risk, pose, to, patient), (indicate...","[['systems', 'methods', 'using', 'machine', 'l...","[[('systems', 'NNS'), ('methods', 'NNS'), ('us...","[[('system', 'NNS'), ('method', 'NNS'), ('use'...","[(use, machine), (predict, risk), (pose, patie...","[(predict, risk), (pose, patient), (use, train...","[(use, machine), (predict, risk), (pose, patie...","[(predict, risk), (pose, patient), (accord, ce..."


In [ ]:
# find the right predicted chunk from verb-noun pattern

def find_true_verb_noun(index, pattern):
    common_tuples = []
    for tuple1 in data[pattern][index]:
        for tuple2 in data['expected_result'][index]:
            if any(word in tuple2 for word in tuple1):
                common_tuples.append(tuple1)
                break
    return common_tuples

data['true_pattern_webb'] = data.apply(lambda row: find_true_verb_noun(row.name, 'pattern_webb'), axis=1)
data['true_pattern_siahaan'] = data.apply(lambda row: find_true_verb_noun(row.name, 'pattern_siahaan'), axis=1)
data['true_pattern_johann'] = data.apply(lambda row: find_true_verb_noun(row.name, 'pattern_johann'), axis=1)
data['true_new_pattern'] = data.apply(lambda row: find_true_verb_noun(row.name, 'new_pattern'), axis=1)
# print(data['true_pattern_johann'][0])

In [ ]:
# find the false predicted chunk from verb-noun pattern

def find_false_verb_noun(index, pattern):
    disjoint_tuples = []
    for tuple1 in data[pattern][index]:
        found_common = False
        for tuple2 in data['expected_result'][index]:
            if any(word in tuple2 for word in tuple1):
                found_common = True
                break
        if not found_common:
            disjoint_tuples.append(tuple1)
    return disjoint_tuples

data['false_pattern_webb'] = data.apply(lambda row: find_false_verb_noun(row.name, 'pattern_webb'), axis=1)
data['false_pattern_siahaan'] = data.apply(lambda row: find_false_verb_noun(row.name, 'pattern_siahaan'), axis=1)
data['false_pattern_johann'] = data.apply(lambda row: find_false_verb_noun(row.name, 'pattern_johann'), axis=1)
data['false_new_pattern'] = data.apply(lambda row: find_false_verb_noun(row.name, 'new_pattern'), axis=1)
# print(data['false_pattern_johann'][0])

In [ ]:
# find the unpredicted chunk from verb-noun pattern

def find_unpredicted_verb_noun(index, pattern):
    unpredicted_tuples = []
    for tuple2 in data['expected_result'][index]:
        found_common = False
        for tuple1 in data[pattern][index]:
            if any(word in tuple1 for word in tuple2):
                found_common = True
                break
        if not found_common:
            unpredicted_tuples.append(tuple2)
    unpredicted_tuples = list(set(unpredicted_tuples))
    return unpredicted_tuples

data['unpredicted_pattern_webb'] = data.apply(lambda row: find_unpredicted_verb_noun(row.name, 'pattern_webb'), axis=1)
data['unpredicted_pattern_siahaan'] = data.apply(lambda row: find_unpredicted_verb_noun(row.name, 'pattern_siahaan'), axis=1)
data['unpredicted_pattern_johann'] = data.apply(lambda row: find_unpredicted_verb_noun(row.name, 'pattern_johann'), axis=1)
data['unpredicted_new_pattern'] = data.apply(lambda row: find_unpredicted_verb_noun(row.name, 'new_pattern'), axis=1)
# print(data['unpredicted_pattern_johann'][0])

In [ ]:
data_patent = data.loc[data.index.isin(list(range(0, 10)) + list(range(20, 30)))]
data_isco = data.loc[data.index.isin(list(range(10, 20)) + list(range(30, 40)))]
data.head()

,data,expected_result,tokenized,pos_tagged_spacy,lemmatized,pattern_webb,pattern_siahaan,pattern_johann,new_pattern,true_pattern_webb,...,true_pattern_johann,true_new_pattern,false_pattern_webb,false_pattern_siahaan,false_pattern_johann,false_new_pattern,unpredicted_pattern_webb,unpredicted_pattern_siahaan,unpredicted_pattern_johann,unpredicted_new_pattern
0,the present invention relates to a method for ...,"[(inputting, target, word), (automatically, ge...","[['present', 'invention', 'relates', 'method',...","[[('present', 'JJ'), ('invention', 'NN'), ('re...","[[('present', 'JJ'), ('invention', 'NN'), ('re...","[(base, word), (mean, clarification), (base, w...","[(base, word), (base, word), (provide, examine...","[(generate, vocabulary, problem), (use, deep, ...","[(generate, vocabulary, problem), (use, deep, ...","[(base, word), (base, word), (give, problem), ...",...,"[(generate, vocabulary, problem), (base, word)...","[(generate, vocabulary, problem), (base, word)...","[(mean, clarification), (mean, clarification),...","[(provide, examinee, student), (investigate, u...","[(use, deep, learning), (mean, clarification),...","[(use, deep, learning), (provide, examinee, st...",[],[],[],[]
1,"the present disclosure is directed to methods,...","[(provide, different, way, to, prepare, food, ...","[['present', 'disclosure', 'directed', 'method...","[[('present', 'JJ'), ('disclosure', 'NN'), ('d...","[[('present', 'JJ'), ('disclosure', 'NN'), ('d...","[(prepare, food), (provide, scalability), (pro...","[(prepare, food, dish, subsystem), (provide, s...","[(include, robotic, kitchen), (provide, differ...","[(include, robotic, kitchen, calibration), (pr...","[(prepare, food), (provide, scalability), (pro...",...,"[(include, robotic, kitchen), (provide, differ...","[(include, robotic, kitchen, calibration), (pr...","[(serve, aid)]",[],"[(meet, operational, requirement), (serve, aid)]","[(meet, operational, requirement)]","[(operate, and, interact, with, the, various, ...","[(operate, and, interact, with, the, various, ...","[(share, task), (a, collaboration, on, share, ...",[]
2,an anti-theft method for an oil tank truck bas...,"[(output, an, alarm, signal, to, a, local, ala...","[['anti-theft', 'method', 'oil', 'tank', 'truc...","[[('anti', 'JJ'), ('-', 'JJ'), ('theft', 'JJ')...","[[('anti', 'JJ'), ('-', 'JJ'), ('theft', 'JJ')...","[(base, machine), (follow, step), (acquire, sc...","[(base, machine, learning, comprises), (follow...","[(base, machine), (follow, step), (acquire, sc...","[(base, machine, learning, comprises), (follow...","[(acquire, scene), (occur, scene), (predict, a...",...,"[(acquire, scene), (occur, scene), (predict, a...","[(acquire, scene, picture, oil, filling, port)...","[(base, machine), (follow, step), (use, machin...","[(base, machine, learning, comprises), (follow...","[(base, machine), (follow, step), (use, machin...","[(base, machine, learning, comprises), (follow...",[],[],[],[]
3,the invention provides an artificial intellige...,"[(categorize, subsequent), (categorize, event)...","[['invention', 'provides', 'artificial', 'inte...","[[('invention', 'NN'), ('provides', 'VBZ'), ('...","[[('invention', 'NN'), ('provide', 'VBZ'), ('a...","[(categorize, event), (categorize, event), (pr...","[(categorize, event), (categorize, event), (pr...","[(categorize, event), (comprise, first, state)...","[(categorize, event), (comprise, first, state)...","[(categorize, event), (categorize, event), (pr...",...,"[(categorize, event), (categorize, event), (ca...","[(categorize, event), (categorize, event), (ca...",[],[],"[(comprise, first, state)]","[(comprise, first, state)]",[],[],[],[]
4,systems and methods for using machine learning...,"[(predict, risk, pose, to, patient), (indicate...","[['systems', 'methods', 'using', 'machine', 'l...","[[('systems', 'NNS'), ('methods', 'NNS'), ('us...","[[('system', 'NNS'), ('method', 'NNS'), ('use'...","[(use, machine), (predict, risk), (pose, patie...","[(predict, risk), (pose, patien

In [ ]:
# count percentage of pattern webb accuracy

def eval_webb(data):
  tp_pattern_webb = data['true_pattern_webb'].apply(len).sum()
  fp_pattern_webb = data['false_pattern_webb'].apply(len).sum()
  fn_pattern_webb = data['unpredicted_pattern_webb'].apply(len).sum()
  precision_pattern_webb = tp_pattern_webb / (tp_pattern_webb + fp_pattern_webb)
  recall_pattern_webb = tp_pattern_webb / (tp_pattern_webb + fn_pattern_webb)
  f1score_pattern_webb = 2 * (precision_pattern_webb * recall_pattern_webb) / (precision_pattern_webb + recall_pattern_webb)

  print("Precision = ", precision_pattern_webb)
  print("Recall = ", recall_pattern_webb)
  print("F1 score = ", f1score_pattern_webb)

print("\nHasil chunk pattern webb all data")
eval_webb(data)
print("\nHasil chunk pattern webb data patent")
eval_webb(data_patent)
print("\nHasil chunk pattern webb data isco")
eval_webb(data_isco)


Hasil chunk pattern webb all data
Precision =  0.6883561643835616
Recall =  0.6813559322033899
F1 score =  0.6848381601362862

Hasil chunk pattern webb data patent
Precision =  0.6792452830188679
Recall =  0.8044692737430168
F1 score =  0.7365728900255755

Hasil chunk pattern webb data isco
Precision =  0.7125
Recall =  0.49137931034482757
F1 score =  0.5816326530612245


In [ ]:
# count percentage of pattern siahaan accuracy

def eval_siahaan(data):
  tp_pattern_siahaan = data['true_pattern_siahaan'].apply(len).sum()
  fp_pattern_siahaan = data['false_pattern_siahaan'].apply(len).sum()
  fn_pattern_siahaan = data['unpredicted_pattern_siahaan'].apply(len).sum()
  precision_pattern_siahaan = tp_pattern_siahaan / (tp_pattern_siahaan + fp_pattern_siahaan)
  recall_pattern_siahaan = tp_pattern_siahaan / (tp_pattern_siahaan + fn_pattern_siahaan)
  f1score_pattern_siahaan = 2 * (precision_pattern_siahaan * recall_pattern_siahaan) / (precision_pattern_siahaan + recall_pattern_siahaan)

  print("Precision = ", precision_pattern_siahaan)
  print("Recall = ", recall_pattern_siahaan)
  print("F1 score = ", f1score_pattern_siahaan)

print("\nHasil chunk pattern siahaan all data")
eval_siahaan(data)
print("\nHasil chunk pattern siahaan data patent")
eval_siahaan(data_patent)
print("\nHasil chunk pattern siahaan data isco")
eval_siahaan(data_isco)


Hasil chunk pattern siahaan all data
Precision =  0.7491039426523297
Recall =  0.7491039426523297
F1 score =  0.7491039426523297

Hasil chunk pattern siahaan data patent
Precision =  0.7587939698492462
Recall =  0.8628571428571429
F1 score =  0.8074866310160428

Hasil chunk pattern siahaan data isco
Precision =  0.725
Recall =  0.5576923076923077
F1 score =  0.6304347826086957


In [ ]:
# count percentage of pattern johann accuracy

def eval_johann(data):
  tp_pattern_johann = data['true_pattern_johann'].apply(len).sum()
  fp_pattern_johann = data['false_pattern_johann'].apply(len).sum()
  fn_pattern_johann = data['unpredicted_pattern_johann'].apply(len).sum()
  precision_pattern_johann = tp_pattern_johann / (tp_pattern_johann + fp_pattern_johann)
  recall_pattern_johann = tp_pattern_johann / (tp_pattern_johann + fn_pattern_johann)
  f1score_pattern_johann = 2 * (precision_pattern_johann * recall_pattern_johann) / (precision_pattern_johann + recall_pattern_johann)

  print("Precision = ", precision_pattern_johann)
  print("Recall = ", recall_pattern_johann)
  print("F1 score = ", f1score_pattern_johann)

print("\nHasil chunk pattern johann all data")
eval_johann(data)
print("\nHasil chunk pattern johann data patent")
eval_johann(data_patent)
print("\nHasil chunk pattern johann data isco")
eval_johann(data_isco)


Hasil chunk pattern johann all data
Precision =  0.7106598984771574
Recall =  0.89171974522293
F1 score =  0.7909604519774012

Hasil chunk pattern johann data patent
Precision =  0.7031802120141343
Recall =  0.943127962085308
F1 score =  0.8056680161943319

Hasil chunk pattern johann data isco
Precision =  0.7297297297297297
Recall =  0.7864077669902912
F1 score =  0.7570093457943925


In [ ]:
# count percentage of new pattern accuracy

def eval_new_pattern(data):
  tp_new_pattern = data['true_new_pattern'].apply(len).sum()
  fp_new_pattern = data['false_new_pattern'].apply(len).sum()
  fn_new_pattern = data['unpredicted_new_pattern'].apply(len).sum()
  precision_new_pattern = tp_new_pattern / (tp_new_pattern + fp_new_pattern)
  recall_new_pattern = tp_new_pattern / (tp_new_pattern + fn_new_pattern)
  f1score_new_pattern = 2 * (precision_new_pattern * recall_new_pattern) / (precision_new_pattern + recall_new_pattern)

  print("Precision = ", precision_new_pattern)
  print("Recall = ", recall_new_pattern)
  print("F1 score = ", f1score_new_pattern)

print("\nHasil chunk new pattern all data")
eval_new_pattern(data)
print("\nHasil chunk new pattern data patent")
eval_new_pattern(data_patent)
print("\nHasil chunk new pattern data isco")
eval_new_pattern(data_isco)


Hasil chunk new pattern all data
Precision =  0.7763496143958869
Recall =  0.9773462783171522
F1 score =  0.8653295128939829

Hasil chunk new pattern data patent
Precision =  0.7833935018050542
Recall =  0.9908675799086758
F1 score =  0.875

Hasil chunk new pattern data isco
Precision =  0.7589285714285714
Recall =  0.9444444444444444
F1 score =  0.8415841584158414


In [ ]:
# hasile

data.to_excel('keyword_extraction_eval_result (4).xlsx', index=False)